In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sentence_transformers import SentenceTransformer

# Carregar os dados
file_path = 'temas_e_perguntas_limpo.csv'
data = pd.read_csv(file_path)

# Passo 1: Garantir divisão com 2/3 para treino por tema
def split_data_with_minimum_proportion(df, test_size=0.33, random_state=42):
    train_data = pd.DataFrame()
    test_data = pd.DataFrame()

    for label in df["Tema"].unique():
        class_data = df[df["Tema"] == label]
        train_class_data, test_class_data = train_test_split(
            class_data, test_size=test_size, random_state=random_state
        )
        train_data = pd.concat([train_data, train_class_data])
        test_data = pd.concat([test_data, test_class_data])

    return train_data, test_data

train_df, test_df = split_data_with_minimum_proportion(data)

# Verificar proporções
train_class_distribution = train_df["Tema"].value_counts()
test_class_distribution = test_df["Tema"].value_counts()

# Passo 2: Gerar embeddings para as perguntas usando Sentence Transformers
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Gerar embeddings
train_embeddings = np.array([model.encode(text) for text in train_df["Pergunta"]])
test_embeddings = np.array([model.encode(text) for text in test_df["Pergunta"]])

# Passo 3: Codificar os rótulos (Temas)
le = LabelEncoder()
y_train = le.fit_transform(train_df["Tema"])
y_test = le.transform(test_df["Tema"])

# Passo 4: Treinar o modelo
classifier = RandomForestClassifier(random_state=42)
classifier.fit(train_embeddings, y_train)

# Passo 5: Avaliar no conjunto de teste
y_pred = classifier.predict(test_embeddings)
report = classification_report(y_test, y_pred, target_names=le.classes_)

# Função para classificar novos textos
def classify_new_question(question, model, classifier, label_encoder):
    embedding = model.encode(question).reshape(1, -1)
    prediction = classifier.predict(embedding)
    return label_encoder.inverse_transform(prediction)[0]

# Prever os temas para todas as questões do conjunto de teste
test_predictions = classifier.predict(test_embeddings)

# Decodificar as previsões para os temas originais
test_predicted_labels = le.inverse_transform(test_predictions)

# Adicionar as previsões ao DataFrame de teste para comparação
test_df["Predicted_Tema"] = test_predicted_labels

# Avaliar o desempenho
print("Relatório de Classificação no Conjunto de Teste:")
print(classification_report(y_test, test_predictions, target_names=le.classes_))

# Mostrar as 10 primeiras linhas com o tema real e o tema previsto
print("Amostra de Classificação no Conjunto de Teste:")
print(test_df[["Pergunta", "Tema", "Predicted_Tema"]].head(10))



Relatório de Classificação no Conjunto de Teste:
                                precision    recall  f1-score   support

                  Agropecuária       0.00      0.00      0.00         4
                   Agrotóxicos       0.57      1.00      0.73         4
                     Atmosfera       0.40      0.50      0.44         4
                         BRICS       0.60      0.60      0.60         5
                  BioGeografia       0.00      0.00      0.00         3
                      Biosfera       0.00      0.00      0.00         4
             Blocos Econômicos       0.00      0.00      0.00         4
    Capitalismo e Globalização       0.00      0.00      0.00         4
                   Cartografia       0.25      0.60      0.35         5
                  Centro-Oeste       0.00      0.00      0.00         4
                         China       0.62      1.00      0.77         5
                   Chuva Ácida       0.40      0.50      0.44         4
              

c:\Users\joaom\AppData\Local\Continuum\anaconda3\envs\pfe\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\joaom\AppData\Local\Continuum\anaconda3\envs\pfe\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\joaom\AppData\Local\Continuum\anaconda3\envs\pfe\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf